# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool
import time

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope(y_axis):
    global data
    ma_len = len(y_axis)
    
    x_axis = []
    for i in range(ma_len):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

In [3]:
def chunk_ticks(df, number_of_ticks):   
    global data
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    tick_avg = []
    spread_avg = []
    tick_sd = []
    
    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])
        
        tick_avg.append(np.mean(tick_list))
        spread_avg.append(np.mean(spread_list))
        tick_sd.append(np.std(tick_list))
        
        
    temp_df['tick_avg'] = tick_avg  
    temp_df['spread_avg'] = spread_avg  
    temp_df['tick_sd'] = tick_sd  
    
    return(temp_df)

In [4]:
def before_sma():
    global data    
    data['ssma_list'].append(val)    
    return()

def after_sma():
    global data
    
    data['ssma_list'].popleft()
    data['ssma_list'].append(val)
    data['sema'] = list(pd.DataFrame(list(data['ssma_list'])).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]
    
    if len(data['sema_ready']) < 2:
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = np.nan

    elif len(data['sema_ready']) > 1:
        data['sema_ready'].popleft()
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = data['sema_ready'][-1] - data['sema_ready'][len(data['sema_ready'])-2]
    
    return()

def before_lma():
    global data    
    data['lsma_list'].append(val)    
    return()

def after_lma():
    global data
    
    data['lsma_list'].popleft()
    data['lsma_list'].append(val)
    data['lema'] = list(pd.DataFrame(list(data['lsma_list'])).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]
    
    if len(data['lema_ready']) < 2:
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = np.nan

    elif len(data['lema_ready']) > 1:
        data['lema_ready'].popleft()
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = data['lema_ready'][-1] - data['lema_ready'][len(data['lema_ready'])-2]
    
    return()

In [5]:
def roll_ma(ma_list):
    global data
    ma_len = len(ma_list)
    sema_val = list(pd.DataFrame(ma_list).ewm(span=ma_len).mean()[0])[ma_len - 1]    
    return(sema_val)

## File paths

In [6]:
year = 2019

data = {}
data['number_of_ticks'] = 120
data['rsi_window'] = 14
data['sma_len'] = 10
data['lma_len'] = 60

diff_col = 'sema'
data['pip_diff'] = 0.00002
#diff_col = 'tick_avg'
#data['pip_diff'] = 0.0003

source_file_path = f'data/tick_{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')

source_file_path : data/tick_2019.csv
chunk_file_path : data\chunk_tick_2019.csv
target_file_path : data\tab_tick_2019.csv


## Read data

In [7]:
%%time
df = pd.read_csv(source_file_path, nrows=1000000)
#df = pd.read_csv(source_file_path)
df.head()

Wall time: 1.6 s


,DateTime,Bid,Ask,Volume
0,20190101 22:02:37.254,1.14598,1.14682,4
1,20190101 22:02:38.590,1.14599,1.14682,2
2,20190101 22:02:39.138,1.14599,1.14684,4
3,20190101 22:02:55.787,1.14598,1.14684,4
4,20190101 22:03:02.060,1.14598,1.14684,4


## Data manipulation

In [8]:
%%time

df = chunk_ticks(df, data['number_of_ticks'])
df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')

df = pd.read_csv(chunk_file_path)
df.head()

100%|█████████████████████████| 8334/8334 [00:06<00:00, 1293.24it/s]


Records : 8334
Wall time: 6.78 s


,tick_avg,spread_avg,tick_sd
0,1.146563,0.000419,0.000151
1,1.146465,0.000548,0.000129
2,1.146288,0.000424,0.000059
3,1.146340,0.000534,0.000070
4,1.146309,0.000471,0.000061


In [9]:
%%time

data['rs_max'] = 1e6

data['ssma_list'] = collections.deque([])
data['lsma_list'] = collections.deque([])
data['sema_ready'] = collections.deque([])
data['lema_ready'] = collections.deque([])
df['sema'] = ''
df['lema'] = ''
df['sema_diff'] = ''
df['lema_diff'] = ''


# RSI -----------------------------
df['diff'] = df['tick_avg'].diff()
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])
df['avg_gain'] = df['gain'].rolling(window=data['rsi_window']).mean()
df['avg_loss'] = df['loss'].rolling(window=data['rsi_window']).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= data['rs_max'], data['rs_max']) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

# Simple Moving Averages ------------------
df['ssma'] = df['tick_avg'].rolling(window=data['sma_len']).mean()
df['ssma_diff'] = df['ssma'].diff()
df['lsma'] = df['tick_avg'].rolling(window=data['lma_len']).mean()
df['lsma_diff'] = df['lsma'].diff()
df['sma_diff'] = df['ssma'] - df['lsma']

df['max_tick'] = df['tick_avg'].rolling(window=data['sma_len']).max()
df['min_tick'] = df['tick_avg'].rolling(window=data['sma_len']).min()

df['max_gap'] = df['max_tick'] -  df['tick_avg']
df['min_gap'] = df['min_tick'] - df['tick_avg']

Wall time: 65 ms


In [10]:
%%time
# Emas ----------------
df['sema'] = df['tick_avg'].rolling(window=data['sma_len']).progress_apply(roll_ma)
df['lema'] = df['tick_avg'].rolling(window=data['lma_len']).progress_apply(roll_ma)

df['sema_diff'] = df['sema'].diff()
df['lema_diff'] = df['lema'].diff()

8325it [00:23, 361.01it/s]
8275it [00:22, 360.72it/s]

Wall time: 46 s


In [11]:
%%time
# Slopes -----------------------------
df['small_sema_slope'] = df['sema'].rolling(window=data['sma_len']).progress_apply(get_slope)
df['long_sema_slope'] = df['sema'].rolling(window=data['lma_len']).progress_apply(get_slope)

8316it [00:07, 1176.32it/s]
8266it [00:07, 1084.88it/s]

Wall time: 14.7 s


In [12]:
%%time

df['ema_diff'] = df['sema'] - df['lema']

# Direction -------------------------
df['direction'] = 'same'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] > data['pip_diff']] = 'increase'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -data['pip_diff']] = 'decrease'

# Remove NaNs ------------------------
del df['gain']
del df['loss']
df = df.dropna()
df = df.reset_index(drop=True)
print(f'Total records : {len(df)}')

df.tail()

Total records : 8266
Wall time: 40 ms


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,small_sema_slope,long_sema_slope,ema_diff,direction
8261,1.152318,0.000033,0.000041,1.152477,1.152893,-0.000053,-0.000020,-0.000053,0.000010,0.000055,...,-7.847222e-07,-0.000352,1.152736,1.152318,0.000418,0.000000,-76.805142,43.795587,-0.000416,decrease
8262,1.152414,0.000030,0.000047,1.152457,1.152878,-0.000021,-0.000015,0.000096,0.000017,0.000052,...,2.759028e-06,-0.000387,1.152715,1.152318,0.000301,-0.000096,-76.314011,38.730051,-0.000421,same
8263,1.152435,0.000031,0.000030,1.152445,1.152865,-0.000012,-0.000013,0.000020,0.000018,0.000046,...,4.218750e-06,-0.000419,1.152662,1.152318,0.000228,-0.000116,-75.174852,32.492401,-0.000420,same
8264,1.152412,0.000033,0.000028,1.152435,1.152850,-0.000010,-0.000014,-0.000023,0.000017,0.000048,...,1.313889e-06,-0.000434,1.152662,1.152318,0.000251,-0.000093,-74.011966,25.127868,-0.000415,same
8265,1.152373,0.000034,0.000026,1.152418,1.152834,-0.000017,-0.000016,-0.000039,0.000017,0.000050,...,-1.618056e-06,-0.000451,1.152662,1.152318,0.000290,-0.000054,-72.841407,16.778654,-0.000416,same


## Write data to csv

In [13]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Wall time: 977 ms


## Print Report

In [14]:
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))
df.head(5)

          counts  percentage
same        2820   34.115654
increase    2773   33.547060
decrease    2673   32.337285


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,small_sema_slope,long_sema_slope,ema_diff,direction
0,1.145559,0.000044,0.000050,1.145750,1.146185,-0.000064,-0.000025,-1.375000e-05,0.000009,0.000053,...,-0.000012,-0.000453,1.146168,1.145559,0.000608,0.000000,-76.309441,-38.719835,-0.000435,decrease
1,1.145520,0.000046,0.000015,1.145690,1.146159,-0.000060,-0.000026,-3.950000e-05,0.000009,0.000055,...,-0.000013,-0.000505,1.146093,1.145520,0.000573,0.000000,-78.574300,-42.431910,-0.000470,decrease
2,1.145550,0.000044,0.000035,1.145649,1.146136,-0.000041,-0.000024,3.037500e-05,0.000012,0.000050,...,-0.000013,-0.000547,1.146093,1.145520,0.000542,-0.000030,-79.737026,-45.894139,-0.000486,decrease
3,1.145609,0.000043,0.000014,1.145628,1.146114,-0.000021,-0.000021,5.887500e-05,0.000016,0.000047,...,-0.000013,-0.000582,1.146019,1.145520,0.000410,-0.000089,-79.940066,-48.922191,-0.000486,same
4,1.145608,0.000040,0.000015,1.145613,1.146094,-0.000015,-0.000020,-8.333333e-07,0.000006,0.000047,...,-0.000012,-0.000611,1.145898,1.145520,0.000290,-0.000088,-79.354435,-51.563441,-0.000481,same
